In [ ]:
import numpy as np
from IPython.display import HTML
import matplotlib.pyplot as plt
import torch
from model import PhiNN
from dataset import LandscapeSimulationDataset
from helpers import jump_function

## Examine training and validation datasets

In [ ]:
datdir_train = "data/model_training_data"
datdir_valid = "data/model_validation_data"
nsims_train = 100
nsims_valid = 30

In [ ]:
train_dataset = LandscapeSimulationDataset(
    datdir_train, nsims_train, 2, 
    transform='tensor', 
    target_transform='tensor'
)

validation_dataset = LandscapeSimulationDataset(
    datdir_valid, nsims_valid, 2, 
    transform='tensor', 
    target_transform='tensor'
)

We can preview an individual training datum, which consists of a distribution of cells $X_0\in\mathbb{R}^{n\times d}$ at time $t_0$, and a subsequent distribution of cells $X_1\in\mathbb{R}^{n\times d}$ at time $t_1$, along with parameters $\vec{p}\in\mathbb{R}^{n_{p}}$, where $n$ is the number of cells, $d$ the dimension of the cells, and $n_p$ the number of parameters that parameterize the signal function $f_{sig}(t,\cdot)$.


In [ ]:
idx = 2700
r = 2
train_dataset.preview(idx, xlims=[-r,r], ylims=[-r,r]);

We can view an animation of a full simulation, which consists of many consecutive time point pairs

In [ ]:
simidx = 9
r = 2
ani = train_dataset.animate(
    simidx, xlims=[-r, r], ylims=[-r,r], 
    show=False
)

In [ ]:
HTML(ani)

## Examine models

In [ ]:
model_dir = "models/model4854215"
model_name = "model4854215_20231026_145206"

loss_hist_train = np.load(f"{model_dir}/training_loss_history.npy")
loss_hist_valid = np.load(f"{model_dir}/validation_loss_history.npy")

ncells = int(np.genfromtxt(f"{model_dir}/ncells.txt"))
sigma = float(np.genfromtxt(f"{model_dir}/sigma.txt"))

best_idx = np.argmin(loss_hist_valid)
model_fpath = f"{model_dir}/{model_name}_{best_idx}"

Plot the training and validation loss histories.

In [ ]:
fig, ax = plt.subplots(1, 1)
ax.plot(1+np.arange(len(loss_hist_train)), loss_hist_train, '.-')
ax.set_xlabel(f"epoch")
ax.set_ylabel(f"loss")
ax.set_title(f"Training Loss")

fig, ax = plt.subplots(1, 1)
ax.plot(1+np.arange(len(loss_hist_valid)), loss_hist_valid, '.-')
ax.set_xlabel(f"epoch")
ax.set_ylabel(f"loss")
ax.set_title(f"Validation Loss")

In [ ]:
f_signal = lambda t, p: jump_function(t, p[...,0], p[...,1:3], p[...,3:])
model = PhiNN(
    ndim=2, nsig=2, f_signal=f_signal,
    ncells=ncells, 
    sigma=sigma,
    sample_cells=True,
)

model.load_state_dict(torch.load(model_fpath, map_location=torch.device('cpu')))
model.eval();

## Plot of the force field

In [ ]:
# State space
r = 100
x = np.linspace(-r, r, 20)
y = np.linspace(-r, r, 20)
xs, ys = np.meshgrid(x, y)
z = np.array([xs.flatten(), ys.flatten()]).T
z = torch.tensor(z, dtype=torch.float32, requires_grad=True)

# Signal function
tcrit = 5
p0 = [0, 0]
p1 = [0, 1]

# Evaluation time
t = 0  

signal_params = np.array([tcrit, *p0, *p1])
signal_params = torch.tensor([signal_params], dtype=torch.float32)

f = model.f(torch.tensor([t], dtype=torch.float32), z, signal_params)
phi = model.phi(z).detach()
grad_phi = -model.grad_phi(torch.tensor([t], dtype=torch.float32), z).detach()

f_arr = f.detach().numpy()
fu, fv = f_arr.T
fnorms = np.sqrt(fu**2 + fv**2)

grad_phi_arr = grad_phi.detach().numpy()
gphiu, gphiv = grad_phi_arr.T
gphinorms = np.sqrt(gphiu**2 + gphiv**2)

# Plot force field generated by the landscape (untilted)
fig, ax = plt.subplots(1, 1)
ax.set_xlabel(f"$x$")
ax.set_ylabel(f"$y$")
ax.set_title(f"$-\\nabla\phi(x,y)$")
sc = ax.quiver(xs, ys, gphiu/gphinorms, gphiv/gphinorms, gphinorms, cmap='jet')
cbar = plt.colorbar(sc)
cbar.ax.set_title("$\|-\\nabla\phi\|$", size=10)
cbar.ax.tick_params(labelsize=8)

# Plot force field, tilted by signals
fig, ax = plt.subplots(1, 1)
ax.set_xlabel(f"$x$")
ax.set_ylabel(f"$y$")
ax.set_title(f"$f(x,y)$")
sc = ax.quiver(xs, ys, fu/fnorms, fv/fnorms, fnorms, cmap='jet')
cbar = plt.colorbar(sc)
cbar.ax.set_title("$\|f\|$", size=10)
cbar.ax.tick_params(labelsize=8)

# Plot Landscape Phi
fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
ax.set_xlabel(f"$x$")
ax.set_ylabel(f"$y$")
ax.set_title(f"$\phi(x,y)$")
ax.plot_surface(xs, ys, phi.reshape(xs.shape), cmap='coolwarm')
ax.view_init(30, -45)

In [ ]:
t0 = 0
t1 = 10
y0 = np.random.rand(ncells, 2)
sigparams = [5, 0, 0, 1, 1]

inputs = (
    torch.tensor([t0], dtype=torch.float32),
    torch.tensor([t1], dtype=torch.float32),
    torch.tensor([y0], dtype=torch.float32, requires_grad=True),
    torch.tensor([sigparams], dtype=torch.float32),
)
z, z_hist = model.simulate_forward(*inputs, dt=1e-1, history=True)

In [ ]:
# State space
r = 4
x = np.linspace(-r, r, 20)
y = np.linspace(-r, r, 20)
xs, ys = np.meshgrid(x, y)
z = np.array([xs.flatten(), ys.flatten()]).T
z = torch.tensor([z], dtype=torch.float32, requires_grad=True)

# Signal function
tcrit = sigparams[0]
p0 = sigparams[1:3]
p1 = sigparams[3:5]
sigparams_tensor = torch.tensor([sigparams], dtype=torch.float32)

tvals = np.linspace(0, 10, len(z_hist))
for i, t in enumerate(tvals):
    if i % 10 != 0:
        continue
    f = model.f(torch.tensor([t]), z, sigparams_tensor)
    phi = model.phi(z).detach()
    f_arr = f.detach().numpy()
    u, v = f_arr.T
    norms = np.sqrt(u**2 + v**2)

    fig, ax = plt.subplots(1, 1, figsize=(6.4, 4.8))
    ax.set_xlabel(f"$x$")
    ax.set_ylabel(f"$y$")
    ax.set_title(f"$f(x,y), t={t:.4f}$")
    sc = ax.quiver(xs, ys, u/norms, v/norms, norms, cmap='jet')
    cbar = plt.colorbar(sc)
    cbar.ax.set_title("$\|f\|$", size=10)
    cbar.ax.tick_params(labelsize=8)
    outdir = "out/snaps"

    ax.scatter(z_hist[i][0,:,0], z_hist[i][0,:,1], s=1)

    plt.savefig(f"{outdir}/frame{i}.png")
    plt.close()

In [ ]:
batch_idx = 0

fig, ax = plt.subplots(1, 1)
changeidx = np.argmin(np.abs(tvals-tcrit))
for idx in range(40):
    ztraj = np.array([zi[batch_idx, idx] for zi in z_hist]).T
    ax.plot(ztraj[0,0], ztraj[1,0], 'r.')
    ax.plot(ztraj[0,changeidx], ztraj[1,changeidx], 'y*')
    ax.plot(ztraj[0,-1], ztraj[1,-1], 'co', markersize=4)
    ax.plot(ztraj[0], ztraj[1], 'k', alpha=0.25)

ax.legend(['start', 'signal change', 'end']);

In [ ]:
# from animator import SimulationAnimator

# simdir = "data/model_training_data/sim0"
# ani = SimulationAnimator(
#     ts=np.load(f"{simdir}/ts.npy"),
#     xys=np.load(f"{simdir}/xs.npy")
# )

# ani.animate('delme');